In [2]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../codes/")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../codes/scpy2/")))

from scpy2.utils.nbmagics import install_magics
install_magics()
del install_magics

D:\Program_File\VSCode_Portable\workon_env\pyElan\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
d:\09_Work\Jun_Workspace\Python_Scientific_Computing_Best_Practices\繁中程式碼\codes\scpy2\utils\program_finder.py:25: UserWarning: Inkscape not found
  warnings.warn("Inkscape not found")


In [3]:
import numpy as np

## 多維陣列的索引存取

### 索引物件

多維陣列的索引應該是一個長度和陣列的維數相同的元組。如果索引元組的長度比陣列的維數大，就會出錯；如果小，就會在索引元組的後面補`":"`，使得它的長度與陣列維數相同。

如果索引物件不是元組，則 numpy 會首先把它轉為元組。這種轉換可能會和使用者所希望的不一致，因此為了避免出現問題，請「顯性」地使用元組作為索引。例如陣列 a 是一個 3D 陣列，下面使用一個二維清單 lidx 和二維陣列 aidx 作為索引，獲得的結果就不一樣。

In [4]:
a = np.arange(3 * 4 * 5).reshape(3, 4, 5)
lidx = [[0], [1]]
aidx = np.array(lidx)
%C a[lidx]; a[aidx]

     a[lidx]                a[aidx]          
-----------------  --------------------------
[[5, 6, 7, 8, 9]]  [[[[ 0,  1,  2,  3,  4],  
                      [ 5,  6,  7,  8,  9],  
                      [10, 11, 12, 13, 14],  
                      [15, 16, 17, 18, 19]]],
                                             
                                             
                    [[[20, 21, 22, 23, 24],  
                      [25, 26, 27, 28, 29],  
                      [30, 31, 32, 33, 34],  
                      [35, 36, 37, 38, 39]]]]


<string>:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


這是因為 numpy 將列表 lidx 轉換成了 `([0],[1])`，而將陣列 aidx 轉換成了 `(aidx,:,:)`

In [5]:
%C a[tuple(lidx)]; a[aidx,:,:]

  a[tuple(lidx)]          a[aidx,:,:]        
-----------------  --------------------------
[[5, 6, 7, 8, 9]]  [[[[ 0,  1,  2,  3,  4],  
                      [ 5,  6,  7,  8,  9],  
                      [10, 11, 12, 13, 14],  
                      [15, 16, 17, 18, 19]]],
                                             
                                             
                    [[[20, 21, 22, 23, 24],  
                      [25, 26, 27, 28, 29],  
                      [30, 31, 32, 33, 34],  
                      [35, 36, 37, 38, 39]]]]


經過各種轉換和增加 `":"` 之後獲得了一個標準的索引元組。它的各個元素有以下幾種類型：切片、整數、整數陣列 和 布林陣列。如果元素不是這些類型，如 列表 或 元組，就將其轉換成 整數陣列。

如果 索引元組 的所有元素都是 切片 和 整數，那麼用它作為索引獲得的是原始陣列的視圖，即它和原始陣列共用資料儲存空間。

### 整數群組作為索引

下面來看索引元組中的元素由切片和整數陣列組成的情況。假設整數陣列有 $N_t$ 個，而切片有 $N_s$ 個。 $N_t + N_s$ 為陣列的維數 D。

首先，這 $N_t$ 個整數陣列必須滿足廣播條件，假設它們進行廣播之後的維數為 M，形狀為 $(d_0, d_1, ..., d_{M-1})$。

如果 $N_s$ 為 0，即沒有切片元素時，則索引所得到的結果陣列 result 的形狀和整數陣列廣播之後的形狀相同。它的每個元素值按照下面的公式獲得：

$result[i_0, i_1, ..., i_{M-1}] = X[ind_0[i_0, i_1, ..., i_{M-1}], ..., ind_{N_t-1}[i_0, i_1, ..., i_{M-1}]]$

其中，$ind_0$ 到 $ind_{N_t-1}$ 為進行廣播之後的整數陣列。讓我們看一個實例，進一步加深對此公式的了解：

> **TIP**

> 若只需要沿著指定軸透過整數群組取得元素，可以使用`numpy.take()`函數，其運算速度比整數群組的索引運算略快，並且支援索引越界處理。

In [6]:
i0 = np.array([[1, 2, 1], [0, 1, 0]])
i1 = np.array([[[0]], [[1]]])
i2 = np.array([[[2, 3, 2]]])
b = a[i0, i1, i2]
b

array([[[22, 43, 22],
        [ 2, 23,  2]],

       [[27, 48, 27],
        [ 7, 28,  7]]])

首先，i0, i1, i2 三個整數陣列的 `shape` 屬性分別為 (2,3), (2,1,1), (1,1,3)。根據廣播規則，先在長度不足 3 的 `shape` 屬性前面補 1，使得它們的維數相同，廣播之後的 `shape` 屬性為各個軸的最大值：
```
(1, 2, 3)   i0.shape (shape長度不足3，前面補 1)
(2, 1, 1)   i1.shape
(1, 1, 3)   i2.shape
---------
 2  2  3    b.shape
```
即三個整數陣列廣播之後的 `shape` 屬性為 (2,2,3)，這也就是索引運算所得到的結果陣列的維數：

In [9]:
b.shape

(2, 2, 3)

我們可以使用 `broadcast_arrays()` 檢視廣播之後的陣列：

In [10]:
ind0, ind1, ind2 = np.broadcast_arrays(i0, i1, i2)
%C ind0; ind1; ind2

     ind0           ind1           ind2    
-------------  -------------  -------------
[[[1, 2, 1],   [[[0, 0, 0],   [[[2, 3, 2], 
  [0, 1, 0]],    [0, 0, 0]],    [2, 3, 2]],
                                           
 [[1, 2, 1],    [[1, 1, 1],    [[2, 3, 2], 
  [0, 1, 0]]]    [1, 1, 1]]]    [2, 3, 2]]]


對於 b 中的任意一個元素 `b[i,j,k]` ，它是陣列 a 中經過 ind0, ind1, ind2 進行索引轉換之後的值：

In [12]:
i, j, k = 0, 1, 2
print( b[i, j, k], a[ind0[i, j, k], ind1[i, j, k], ind2[i, j, k]] )

i, j, k = 1, 1, 1
print( b[i, j, k], a[ind0[i, j, k], ind1[i, j, k], ind2[i, j, k]] )

2 2
28 28


下面考慮 $N_s$ 不為 0 的情況。當存在切片索引時，情況就變得更加複雜了。  
可以細分為兩種情況：索引元組中的整數陣列之間沒有切片，即整數陣列只有一個或連續的多個整數陣列。  
這時結果陣列的 `shape` 屬性為：將原始陣列的 `shape` 屬性中整數陣列所佔據的部分取代為它們廣播之後的 `shape` 屬性。  
例如假設原始陣列 a 的 `shape` 屬性為 (3,4,5)，i0 和 i1 廣播之後的形狀為 (2,2,3)，則 `a[1:3, i0, i1]` 的形狀為 (2,2,2,3)。
```
(1, 2, 3)   i0.shape (shape長度不足3，前面補 1)
(2, 1, 1)   i1.shape
---------
 2  2  3    i0, i1 廣播之後的shape
 ```

In [14]:
print(a.shape)
print(i0.shape)
print(i1.shape)
c = a[1:3, i0, i1]
c.shape

(3, 4, 5)
(2, 3)
(2, 1, 1)


(2, 2, 2, 3)

其中，c 的 `shape` 屬性中的第一個 2 是切片 "1:3" 的長度，後面的 (2,2,3) 則是 i0 和 i1 廣播之後的陣列的形狀：

In [15]:
ind0, ind1 = np.broadcast_arrays(i0, i1)
ind0.shape

(2, 2, 3)

In [17]:
i, j, k = 1, 1, 2
print( c[:, i, j, k] )
print( a[1:3, ind0[i, j, k], ind1[i, j, k]] )  # 和c[:,i,j,k]的值相同

[21 41]
[21 41]


當索引元組中的整數陣列不連續時，結果陣列的 `shape` 屬性為整數陣列廣播之後的形狀後面增加上切片元素所對應的形狀。例如 `a[i0,:,i1]` 的 `shape` 屬性為 (2,2,3,4)，其中(2,2,3) 是i0和i1 廣播之後的形狀，而 4 是陣列 a 的第 1 軸的長度：

In [18]:
d = a[i0, :, i1]
d.shape

(2, 2, 3, 4)

In [19]:
i, j, k = 1, 1, 2
%C d[i,j,k,:]; a[ind0[i,j,k],:,ind1[i,j,k]]

   d[i,j,k,:]     a[ind0[i,j,k],:,ind1[i,j,k]]
----------------  ----------------------------
[ 1,  6, 11, 16]  [ 1,  6, 11, 16]            


### 一個復雜的實例

下面使用所學的索引存取的知識，解決在 numpy 郵寄清單中提出的比較經典的問題。

> **LINK**  
> http://mail.scipy.org/pipermail/numpy-discussion/2008-July/035764.html  
> NumPy信件清單中的原文連結

對問題進行一些簡化，提問者想要實現的索引運算是：有一個形狀為 (I, J, K) 的 3D 陣列 v 和一個形狀為  (I, J) 的二維陣列 idx，idx 的每個值都是 0 到 K-L的整數。他想透過索引運算獲得一個陣列 r，對於第 0 軸和第 1 軸的每個索引 i 和 j 都滿足下面的條件：
```
r[i,j,:]=v[i,j,idx[i,j]:idx[i,j]+L]
```
如圖2-7 所示，左圖中不透明的方塊是我們希望取得的部分，透過索引運算之後將獲得右側所示的陣列。

![](2022-03-21-09-55-57.png)

圖2-7 3D陣列索引運算問題的示意圖

首先建立一個方便偵錯的陣列 v，它在第 2 軸上每一層的值就是該層的高度，即 `v[:,:,i]` 的所有的元素值都為 i。然後隨機產生陣列 idx，它的每個元素的設定值都在 0 到 K-L之間：

In [20]:
#%hide
%exec_python -m scpy2.numpy.array_index_demo

In [22]:
I, J, K, L = 6, 7, 8, 3
_, _, v = np.mgrid[:I, :J, :K]
idx = np.random.randint(0, K - L, size=(I, J))

然後用陣列 idx 建立第 2 軸的索引陣列 idx_k ，它是一個形狀為 (I,J,L)的 3D 陣列。它的第 2 軸上的每一層的值都等於 idx 陣列加上層的高度，即 `idx_k[:,:,i]=idx[:,:]+i`：

In [23]:
idx_k = idx[:, :, None] + np.arange(3)
idx_k.shape

(6, 7, 3)

然後分別建立第 0 軸和第 1 軸的索引陣列，它們的 `shape` 分別為 (1,1,1) 和 (1,J,1)。

In [24]:
idx_i, idx_j, _ = np.ogrid[:I, :J, :K]

使用 idx_i, idx_j, idx_k 對陣列 v 進行索引運算即可獲得結果：

In [25]:
r = v[idx_i, idx_j, idx_k]    
i, j = 2, 3  # 驗證結果，讀者可以將之修改為使用循環驗證所有的元素
%C r[i,j,:]; v[i,j,idx[i,j]:idx[i,j]+L]

 r[i,j,:]  v[i,j,idx[i,j]:idx[i,j]+L]
---------  --------------------------
[3, 4, 5]  [3, 4, 5]                 


### 布爾陣列作索引

當使用布林陣列直接作為索引物件或元組索引物件中有布林陣列時，都相當於用 `nonzero()` 將布林陣列轉換成一組整數陣列，然後使用整數陣列進行索引運算。

`nonzero(a)` 傳回陣列 a 中值不為零的元素的索引，它的傳回值是一個長度為 `a.ndim`(陣列 a 的軸數) 的元組，元組的每個元素都是一個整數陣列，其值為非零元素的索引在對應軸上的值。例如對於一維布林陣列 b1，`nonzero(a)` 所得到的是一個長度為 1 的元組，它表示 `b1[0]` 和 `b1[2]` 的值不為 0。

> **TIP**

> 若只需要沿著指定軸透過布爾陣列取得元素，可以使用`numpy.compress()`函數。

In [18]:
b1 = np.array([True, False, True, False])
np.nonzero(b1)

(array([0, 2]),)

對於二維陣列 b2，`nonzero(a)`所得到的是一個長度為 2 的元組。它的第 0 個元素是陣列 a 中值不為 0 的元素的第 0 軸的索引，第 1 個元素則是第 1 軸的索引，因此從下面的結果可知 `b2[0,0]`, `b2[0,2]`, `b2[1,0]` 的值不為 0：

In [26]:
b2 = np.array([[True, False, True], [True, False, False]])
np.nonzero(b2)

(array([0, 0, 1], dtype=int64), array([0, 2, 0], dtype=int64))

當布林陣列直接作為索引時，相當於使用由 `nonzero()` 轉換之後的元組作為索引物件：

In [28]:
a = np.arange(3 * 4 * 5).reshape(3, 4, 5)
# %C a[b2]; a[np.nonzero(b2)]
print(a[b2])
print(a[np.nonzero(b2)])

IndexError: boolean index did not match indexed array along dimension 0; dimension is 3 but corresponding boolean dimension is 2

當索引物件是元組，並且其中有布林陣列時，相當於將布林陣列展開為由 `nonzero()` 轉換之後的各個整數陣列：

In [29]:
%C a[1:3, b2]; a[1:3, np.nonzero(b2)[0], np.nonzero(b2)[1]]

IndexError: boolean index did not match indexed array along dimension 1; dimension is 4 but corresponding boolean dimension is 2